### Lab #2 과제 정답

In [ ]:
# 필요한 라이브러리 임포트 (재실행용)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score
import plotly.express as px
import plotly.graph_objects as go

# 데이터 준비 (재실행용)
url = 'https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv'
df = pd.read_csv(url)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)
df['Churn'] = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)
X = df.drop(['customerID', 'Churn'], axis=1)
y = df['Churn']
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 모델 훈련 및 예측 (재실행용)
preprocessor = ColumnTransformer(transformers=[('num', StandardScaler(), numeric_features),('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)])
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),('classifier', LogisticRegression(random_state=42))])
model_pipeline.fit(X_train, y_train)
y_pred = model_pipeline.predict(X_test)
y_scores = model_pipeline.predict_proba(X_test)[:, 1]

In [ ]:
# [문제 1 정답] 혼동 행렬
cm = confusion_matrix(y_test, y_pred)

fig_cm = px.imshow(cm, text_auto=True,
                  labels=dict(x="Predicted", y="True"),
                  x=['Not Churn', 'Churn'], y=['Not Churn', 'Churn'],
                  title="Churn Prediction Confusion Matrix")
fig_cm.show()

In [ ]:
# [문제 2 정답] 평가지표 계산
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

#### [문제 3 정답] 결과 분석 질문
1.  **정답**: 섣불리 판단해서는 안 된다. 이 데이터는 '이탈 안 함'(Not Churn) 클래스가 약 73.5%를 차지하는 불균형 데이터이다. 만약 모델이 모든 고객을 '이탈 안 함'으로 예측해도 정확도는 73.5%가 나온다. 현재 모델의 정확도 80.4%는 이 기준점(baseline)보다는 높지만, 이탈 고객(Positive Class)을 얼마나 잘 예측하는지는 알려주지 못하므로 정확도만으로 성능을 판단하는 것은 위험하다.

2.  **정답**: **정밀도(Precision)**를 더 중요하게 고려해야 한다. 
    * **FP (False Positive)**: 이탈하지 않을 고객을 이탈할 것이라고 잘못 예측. 이 고객에게 불필요한 할인 쿠폰을 발송하게 되어 **마케팅 비용이 낭비**된다.
    * **FN (False Negative)**: 이탈할 고객을 이탈하지 않을 것이라고 잘못 예측. 이 고객은 아무런 조치를 받지 못하고 그대로 **이탈하게 되어 회사의 손실**로 이어진다. 
    * 캠페인의 목적이 '이탈 가능성이 높은 고객'에게 **효율적으로** 접근하는 것이므로, 불필요한 비용 낭비(FP)를 줄이는 것이 우선이다. 따라서 모델이 '이탈한다'고 예측했을 때 실제로 이탈할 확률(정밀도)이 높은 것이 중요하다. 물론 재현율도 중요하지만, 한정된 예산 하에서는 정밀도를 우선하는 것이 합리적인 전략이다.

In [ ]:
# [문제 4 정답] ROC Curve 및 AUC
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
auc_score = roc_auc_score(y_test, y_scores)

fig_roc = go.Figure()
fig_roc.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f'ROC Curve (AUC = {auc_score:.4f})'))
fig_roc.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random (AUC = 0.5)', line=dict(dash='dash')))
fig_roc.update_layout(title='ROC Curve for Churn Prediction',
                  xaxis_title='False Positive Rate',
                  yaxis_title='True Positive Rate')
fig_roc.show()

#### [문제 5 정답] ROC/AUC 분석 질문
**정답**: AUC 점수가 약 0.846으로, 이는 0.5(랜덤 추측)보다는 훨씬 높고 1.0(완벽한 분류)에 가까운 수치이다. 따라서 이 모델은 **'준수한(decent) 혹은 좋은(good)' 성능**을 가진다고 평가할 수 있다. 즉, 모델이 이탈 고객과 비이탈 고객을 상당히 잘 구별해내는 능력을 갖추고 있음을 의미한다.